## Calculate annual statistic per time seres (for each pixel) within a single Equi7grid tile

In [1]:
import os

In [2]:
# Changing the current working directory
os.chdir('/home/return-mmilenkovic/mm_tools/use-case-return')

In [3]:
import numpy as np
import os, osr, glob
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import pandas as pd
import xarray as xr
import rioxarray
import zarr
# import TUW packages
from yeoda.products.preprocessed import SIG0DataCube
from geopathfinder.naming_conventions.yeoda_naming import YeodaFilename
#
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import my aux functions
from auxilary_ts_tools_mm import plot_TS_fromPandaSeres, features_from_S1_TS, features_as_xrrray_ufunc

/home/return-mmilenkovic/miniconda3/envs/yeoda/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [4]:
myTile = 'E057N072T3'
my_chunk_size = 250
my_tile_size = 15000
#
my_row = 3
my_col = 59

Specify the folder (Equi7 tile) with S1 data (a 300x300 km<sup>2</sup>)

In [5]:
tile_dir1_path = r'/project/return/Share/EODC_SA020M/V01R01/' + myTile
tile_dir2_path = r'/project/return/Share/EODC_SA020M/V1M0R1/' + myTile
# specify other parameters:
dimensions=['time', 'band', 'extra_field', 'sensor_field', 'data_version']
# get the filenames
filepaths1 = glob.glob(os.path.join(tile_dir1_path,'*.tif'))
filepaths2 = glob.glob(os.path.join(tile_dir2_path,'*.tif'))

Get the lists of all S1 images in the two eodc folders with S1 data:

In [6]:
tile_names1 = [os.path.basename(aa) for aa in glob.glob(r'/project/return/Share/EODC_SA020M/V01R01/*')]
tile_names2 = [os.path.basename(aa) for aa in glob.glob(r'/project/return/Share/EODC_SA020M/V1M0R1/*')]

In [7]:
# just a small check:
if len(tile_names1) == len(tile_names2):
    print('The number of files is identical')
else:
    print('File number not identical')

The number of files is identical


Read the datacube:

In [8]:
sig0_dc1 = SIG0DataCube(filepaths=filepaths1, dimensions=dimensions, filename_class=YeodaFilename, sres=20, continent='SA')
sig0_dc2 = SIG0DataCube(filepaths=filepaths2, dimensions=dimensions, filename_class=YeodaFilename, sres=20, continent='SA')
# get info:
sig0_dc2.inventory[dimensions].head(5)

,time,band,extra_field,sensor_field,data_version,var_name,tile_name
0,2022-05-25 10:43:21,VV,D069,S1AIWGRDH,V1M0R1,SIG0,E057N072T3
1,2018-05-11 10:34:51,VH,D171,S1BIWGRDH,V1M0R1,SIG0,E057N072T3
2,2022-08-29 10:43:52,VH,D069,S1AIWGRDH,V1M0R1,SIG0,E057N072T3
3,2017-09-01 10:34:51,VV,D171,S1BIWGRDH,V1M0R1,SIG0,E057N072T3
4,2018-12-18 10:43:10,VV,D069,S1BIWGRDH,V1M0R1,SIG0,E057N072T3


Filter by date:

In [9]:
#toi_start, toi_end = datetime(2017, 1, 1), datetime(2023, 1, 1)
toi_start, toi_end = datetime(2017, 1, 1), datetime(2021, 1, 1)
sig0_dc1 = sig0_dc1.filter_by_dimension([(toi_start, toi_end)], [(">=", "<")], name="time", inplace=True)
sig0_dc2 = sig0_dc2.filter_by_dimension([(toi_start, toi_end)], [(">=", "<")], name="time", inplace=True)

Select bands:

In [10]:
sig0_vv_dc1 = sig0_dc1.filter_by_dimension('VV', name='band')
sig0_vh_dc1 = sig0_dc1.filter_by_dimension('VH', name='band')
#
sig0_vv_dc2 = sig0_dc2.filter_by_dimension('VV', name='band')
sig0_vh_dc2 = sig0_dc2.filter_by_dimension('VH', name='band')

Merge and sort the datacubes:

In [11]:
sig0_vv_dc = sig0_vv_dc1.unite(sig0_vv_dc2)
sig0_vv_dc = sig0_vv_dc.sort_by_dimension('time', ascending=True)
#
sig0_vh_dc = sig0_vh_dc1.unite(sig0_vh_dc2)
sig0_vh_dc = sig0_vh_dc.sort_by_dimension('time', ascending=True)
#
sig0_vv_dc.inventory

,filepath,time,band,extra_field,sensor_field,data_version,var_name,tile_name
725,/project/return/Share/EODC_SA020M/V1M0R1/E057N...,2017-01-04 10:33:54,VV,D171,S1BIWGRDH,V1M0R1,SIG0,E057N072T3
1112,/project/return/Share/EODC_SA020M/V1M0R1/E057N...,2017-01-04 10:34:19,VV,D171,S1BIWGRDH,V1M0R1,SIG0,E057N072T3
836,/project/return/Share/EODC_SA020M/V1M0R1/E057N...,2017-01-04 10:34:44,VV,D171,S1BIWGRDH,V1M0R1,SIG0,E057N072T3
986,/project/return/Share/EODC_SA020M/V1M0R1/E057N...,2017-01-09 23:03:36,VV,A076,S1BIWGRDH,V1M0R1,SIG0,E057N072T3
800,/project/return/Share/EODC_SA020M/V1M0R1/E057N...,2017-01-09 23:04:01,VV,A077,S1BIWGRDH,V1M0R1,SIG0,E057N072T3
...,...,...,...,...,...,...,...,...
532,/project/return/Share/EODC_SA020M/V01R01/E057N...,2020-12-31 10:42:57,VV,D069,S1B,V01R01,SIG0,E057N072T3
172,/project/return/Share/EODC_SA020M/V01R01/E057N...,2020-12-31 10:43:22,VV,D069,S1B,V01R01,SIG0,E057N072T3
602,/project/return/Share/EODC_SA020M/V01R01/E057N...,2020-12-31 23:04:03,VV,A076,S1B,V01R01,SIG0,E057N072T3
576,/project/return/Share/EODC_SA020M/V01R01/E057N...,2020-12-31 23:04:28,VV,A077,S1B,V01R01,SIG0,E057N072T3


Select only descending orbits

Specify indexing for looping trough individual chunks within the Equi7grid tile:

In [22]:
desc_list_vh = [aa for aa in sig0_vh_dc.inventory.extra_field.unique().tolist() if aa[0]=='D']
desc_list_vh

['D069', 'D171']

In [23]:
desc_list_vv = [aa for aa in sig0_vv_dc.inventory.extra_field.unique().tolist() if aa[0]=='D']
desc_list_vv

['D171', 'D069', 'D098']

In [25]:
desc_list = list(set(desc_list_vh) & set(desc_list_vv))
desc_list

['D171', 'D069']

In [ ]:
myOrbit = 'D069'

In [ ]:
sig0_vv_orbit_dc = sig0_vv_dc.filter_by_dimension(myOrbit, name="extra_field")
sig0_vv_orbit_dc = sig0_vv_orbit_dc.sort_by_dimension('time', ascending=True)
#
sig0_vh_orbit_dc = sig0_vh_dc.filter_by_dimension(myOrbit, name="extra_field")
sig0_vh_orbit_dc = sig0_vh_orbit_dc.sort_by_dimension('time', ascending=True)
#
sig0_vv_orbit_dc.inventory.head()

In [ ]:
remove_broken_files(sig0_vh_orbit_dc.inventory)

In [ ]:
[os.path.basename(aa) for aa in sig0_vh_orbit_dc.inventory.filepath]

Load data 

In [ ]:
# read a single orbit
sig0_vv_dc_chunk1 = sig0_vv_orbit_dc.load_by_pixels((my_row-1)*my_chunk_size, (my_col-1)*my_chunk_size, row_size=250, col_size=250, dtype='xarray')
sig0_vh_dc_chunk1 = sig0_vh_orbit_dc.load_by_pixels((my_row-1)*my_chunk_size, (my_col-1)*my_chunk_size, row_size=250, col_size=250, dtype='xarray')

Rename the variavle

In [ ]:
sig0_vv_dc_chunk1 = sig0_vv_dc_chunk1.rename({'1':'sig0_vv'})

In [ ]:
sig0_vh_dc_chunk1 = sig0_vh_dc_chunk1.rename({'1':'sig0_vh'})

In [ ]:
# print(sig0_vh_dc_chunk1)

Rescale the data in 2019 and 2020

In [ ]:
sig0_vv_dc_chunk1['sig0_vv'].loc[slice('2019-1-1','2021-1-1'), :, :] = sig0_vv_dc_chunk1.sel(time=slice('2019-1-1','2021-1-1')).apply(lambda x: np.round(x/10.,1)).sig0_vv.values
sig0_vh_dc_chunk1['sig0_vh'].loc[slice('2019-1-1','2021-1-1'), :, :] = sig0_vh_dc_chunk1.sel(time=slice('2019-1-1','2021-1-1')).apply(lambda x: np.round(x/10.,1)).sig0_vh.values

In [ ]:
sig0_vv_dc_chunk1['sig0_vv'].isnull().values.all()

In [ ]:
sig0_vh_dc_chunk1.sig0_vh.sel(time=slice('2017-01-01', '2018-01-01')).mean('time')

In [ ]:
sig0_vh_dc_chunk1.sig0_vh.sel(time=slice('2017-01-01', '2018-01-01')).mean('time').plot(cmap='Greys_r')

In [ ]:
%matplotlib inline
#sig0_vh_dc_chunk1.sig0_vh.min(dim='time').plot(cmap='Greys_r')
sig0_vh_dc_chunk1.sig0_vh.min(dim='time').plot(cmap='Greys_r')

# Inspect Single orbit time-seres

convert to panda seres:

In [ ]:
sig0_vv_ts = sig0_vv_dc_chunk1.sig0_vv.isel(x=249, y=249).dropna(dim='time').to_series()
#sig0_vv_ts = sig0_vv_dc_chunk1.sig0_vv.sel(x=7911673, y=6710188, method="nearest").dropna(dim='time').to_series()

In [ ]:
my_xticks = pd.date_range(datetime(2017,1,1), datetime(2021,1,1), freq='YS')
sig0_vv_ts.plot(style='ro-', xticks=my_xticks, grid=True, figsize=(14,4), legend=True, xlabel='Time', ylabel='Bacscatter Intensity [dB]')

Round the time and resample to 6 day TS:

In [ ]:
sig0_vv_ts.index = sig0_vv_ts.index.round('D')
#
sig0_vv_ts_6d = sig0_vv_ts.resample('6D').interpolate(method='linear')

In [ ]:
sig0_vv_ts_6d.plot(style='bo-',grid=True, figsize=(14,4), legend=True, xlabel='Time', ylabel='Bacscatter Intensity [dB]')

In [ ]:
#%matplotlib widget
%matplotlib inline
plot_TS_fromPandaSeres(sig0_vv_ts_6d)